In [ ]:
!pip install -q diffusers transformers gradio accelerate torch scipy

import torch
from diffusers import StableDiffusionPipeline
import gradio as gr
from PIL import Image
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
use_fp16 = device == "cuda"

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16 if use_fp16 else torch.float32
)
pipe = pipe.to(device)
pipe.safety_checker = None  # Disable safety checker for unrestricted creative output

@torch.no_grad()
def generate_blend(prompt_a, prompt_b, alpha=0.5, guidance_scale=7.5, seed=42):
    """
    Generate an image by blending two prompts using classifier-free guidance.

    Args:
        prompt_a (str): First prompt (e.g., "a gothic cathedral at sunrise").
        prompt_b (str): Second prompt (e.g., "a futuristic cyberpunk city").
        alpha (float): Blend factor between prompts [0.0 (A only) to 1.0 (B only)].
        guidance_scale (float): Classifier-free guidance strength.
        seed (int): Random seed for reproducibility.

    Returns:
        PIL.Image: The generated image based on the blended prompt.
    """

    generator = torch.Generator(device=device).manual_seed(seed)

    # Tokenize both prompts to fixed length for embedding compatibility
    max_length = pipe.tokenizer.model_max_length
    input_a = pipe.tokenizer(prompt_a, return_tensors="pt", padding="max_length", truncation=True, max_length=max_length).input_ids.to(device)
    input_b = pipe.tokenizer(prompt_b, return_tensors="pt", padding="max_length", truncation=True, max_length=max_length).input_ids.to(device)
    input_uncond = pipe.tokenizer([""], return_tensors="pt", padding="max_length", max_length=max_length).input_ids.to(device)

    # Get text embeddings from the encoder
    embedding_a = pipe.text_encoder(input_a)[0]
    embedding_b = pipe.text_encoder(input_b)[0]
    uncond_embedding = pipe.text_encoder(input_uncond)[0]

    blended_embedding = (1 - alpha) * embedding_a + alpha * embedding_b

    text_embeddings = torch.cat([uncond_embedding, blended_embedding], dim=0)

    latents = torch.randn(
        (1, pipe.unet.in_channels, 64, 64),
        generator=generator,
        device=device,
        dtype=torch.float16 if use_fp16 else torch.float32
    ) * pipe.scheduler.init_noise_sigma

    pipe.scheduler.set_timesteps(20, device=device)

    for t in pipe.scheduler.timesteps:
        latent_input = torch.cat([latents] * 2)
        latent_input = pipe.scheduler.scale_model_input(latent_input, t)

        noise_pred = pipe.unet(latent_input, t, encoder_hidden_states=text_embeddings).sample
        noise_uncond, noise_cond = noise_pred.chunk(2)
        noise = noise_uncond + guidance_scale * (noise_cond - noise_uncond)

        latents = pipe.scheduler.step(noise, t, latents).prev_sample

    latents = latents / 0.18215
    image = pipe.vae.decode(latents).sample[0]
    image = (image / 2 + 0.5).clamp(0, 1)  # Normalize to [0, 1]
    image = image.cpu().permute(1, 2, 0).numpy()  # CHW to HWC
    image = (image * 255).astype(np.uint8)

    return Image.fromarray(image)

gr.Interface(
    fn=generate_blend,
    inputs=[
        gr.Textbox(label="Prompt A", value="a gothic cathedral at sunrise"),
        gr.Textbox(label="Prompt B", value="a futuristic cyberpunk city"),
        gr.Slider(0, 1, value=0.5, step=0.05, label="Blend (alpha)"),
        gr.Slider(1, 15, value=7.5, step=0.5, label="Guidance Scale"),
        gr.Slider(0, 10000, value=42, step=1, label="Seed"),
    ],
    outputs=gr.Image(type="pil", label="Blended Output"),
    title="🎨 Prompt Blend with Stable Diffusion",
    description="Blend two prompts using classifier-free guidance in Stable Diffusion. Adjust the alpha slider to control blending.",
).launch(debug=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2c5bafecaebe1a3e10.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<ipython-input-1-48f52b02dea8>:52: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.in_channels'.
  (1, pipe.unet.in_channels, 64, 64),
